# FootOdds Data Preprocessing

The purpose of this Notebooks is to prepare a dataset for modeling.
Source files are collected on internet as csv files by season. The scope is bounded to english PL.

The output is a single files with features and target variable (1/N/2) for all observations.

In [28]:
import pyforest
from sklearn.preprocessing import OrdinalEncoder
from sklearn.impute import SimpleImputer

In [29]:
import os
files_and_directories = os.listdir("data/")
all_files = []
for file in files_and_directories:
    if 'csv' in file:
        temp = pd.read_csv('data/'+str(file))
        all_files.append(temp)
        
data = pd.concat(all_files, sort=False)

data['Date'] = pd.to_datetime(data['Date'])

print(data.shape)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

...

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

(7610, 80)


In [30]:
df = data.copy()
# transform OHE the FTR (full time 1/N/2) and the Half Time (1/N/2)
df.reset_index(inplace=True, drop=True)
temp_ohe = pd.get_dummies(df[['FTR','HTR']])
df = df.merge(temp_ohe, left_index=True, right_index=True)
df.drop(['Div'], axis=1, inplace=True)

# encode the refereee
si = SimpleImputer(fill_value="Unknown", missing_values=np.nan, strategy="constant")
oe = OrdinalEncoder()
referee = si.fit_transform(df[['Referee']])
df['Referee']=referee


referee = oe.fit_transform(df[['Referee']])
df['Referee']=referee


df.dropna(axis=1, how='all', inplace=True)
df.shape

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

(7610, 85)

In [31]:
df.head()

,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,HTR,Referee,...,SBA,0,1,2,FTR_A,FTR_D,FTR_H,HTR_A,HTR_D,HTR_H
0,2018-10-08,Man United,Leicester,2.0,1.0,H,1.0,0.0,H,1.0,...,NaN,NaN,NaN,NaN,0,0,1,0,0,1
1,2018-11-08,Bournemouth,Cardiff,2.0,0.0,H,1.0,0.0,H,11.0,...,NaN,NaN,NaN,NaN,0,0,1,0,0,1
2,2018-11-08,Fulham,Crystal Palace,0.0,2.0,A,0.0,1.0,A,17.0,...,NaN,NaN,NaN,NaN,1,0,0,1,0,0
3,2018-11-08,Huddersfield,Chelsea,0.0,3.0,A,0.0,2.0,A,5.0,...,NaN,NaN,NaN,NaN,1,0,0,1,0,0
4,2018-11-08,Newcastle,Tottenham,1.0,2.0,A,1.0,2.0,A,15.0,...,NaN,NaN,NaN,NaN,1,0,0,1,0,0


In [42]:
missing=pd.DataFrame(df.isnull().sum())
missing.columns = ['cnt']
cols_del=missing.query('cnt>100').sort_values('cnt', ascending=False).index.tolist()
try:
    df.drop(cols_del,axis=1,inplace=True)
except:
    pass

print(f"df Size before dropping some rows {df.shape}")

df.dropna(axis=0, how='any', inplace=True)
print(f"df Size without missing valuesrows {df.shape}, number of np.nan value = {df.isnull().sum().sum()}")



<IPython.core.display.Javascript object>

df Size before dropping some rows (7576, 61)
df Size without missing valuesrows (7576, 61), number of np.nan value = 0


In [56]:
df.dtypes

Date        datetime64[ns]
HomeTeam            object
AwayTeam            object
FTHG               float64
FTAG               float64
                 ...      
FTR_D                uint8
FTR_H                uint8
HTR_A                uint8
HTR_D                uint8
HTR_H                uint8
Length: 61, dtype: object

In [57]:
df

,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,HTR,Referee,...,BbMxAHH,BbAvAHH,BbMxAHA,BbAvAHA,FTR_A,FTR_D,FTR_H,HTR_A,HTR_D,HTR_H
0,2018-10-08,Man United,Leicester,2.0,1.0,H,1.0,0.0,H,1.0,...,1.75,1.70,2.29,2.21,0,0,1,0,0,1
1,2018-11-08,Bournemouth,Cardiff,2.0,0.0,H,1.0,0.0,H,11.0,...,2.20,2.13,1.80,1.75,0,0,1,0,0,1
2,2018-11-08,Fulham,Crystal Palace,0.0,2.0,A,0.0,1.0,A,17.0,...,2.18,2.11,1.81,1.77,1,0,0,1,0,0
3,2018-11-08,Huddersfield,Chelsea,0.0,3.0,A,0.0,2.0,A,5.0,...,1.84,1.80,2.13,2.06,1,0,0,1,0,0
4,2018-11-08,Newcastle,Tottenham,1.0,2.0,A,1.0,2.0,A,15.0,...,2.20,2.12,1.80,1.76,1,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7605,2010-05-09,Everton,Portsmouth,1.0,0.0,H,0.0,0.0,D,25.0,...,1.10,1.06,10.00,7.72,0,0,1,0,1,0
7606,2010-05-09,Hull,Liverpool,0.0,0.0,D,0.0,0.0,D,1.0,...,5.00,4.38,1.20,1.18,0,1,0,0,1,0
7607,2010-05-09,Man United,Stoke,4.0,0.0,H,2.0,0.0,H,16.0,...,2.08,2.02,1.91,1.85,0,0,1,0,0,1
7608,2010-05-09,West Ham,Man City,1.0,1.0,D,1.0,1.0,D,9.0,...,3.25,2.99,1.44,1.35,0,1,0,0,1,0


# Data definition
http://www.football-data.co.uk/notes.txt

### Key to results data:

- Div = League Division
- Date = Match Date (dd/mm/yy)
- Time = Time of match kick off
- HomeTeam = Home Team
- AwayTeam = Away Team
- FTHG and HG = Full Time Home Team Goals
- FTAG and AG = Full Time Away Team Goals
- FTR and Res = Full Time Result (H=Home Win, D=Draw, A=Away Win)
- HTHG = Half Time Home Team Goals
- HTAG = Half Time Away Team Goals
- HTR = Half Time Result (H=Home Win, D=Draw, A=Away Win)

### Match Statistics (where available)
- Attendance = Crowd Attendance
- Referee = Match Referee
- HS = Home Team Shots
- AS = Away Team Shots
- HST = Home Team Shots on Target
- AST = Away Team Shots on Target
- HHW = Home Team Hit Woodwork
- AHW = Away Team Hit Woodwork
- HC = Home Team Corners
- AC = Away Team Corners
- HF = Home Team Fouls Committed
- AF = Away Team Fouls Committed
- HFKC = Home Team Free Kicks Conceded
- AFKC = Away Team Free Kicks Conceded
- HO = Home Team Offsides
- AO = Away Team Offsides
- HY = Home Team Yellow Cards
- AY = Away Team Yellow Cards
- HR = Home Team Red Cards
- AR = Away Team Red Cards
- HBP = Home Team Bookings Points (10 = yellow, 25 = red)
- ABP = Away Team Bookings Points (10 = yellow, 25 = red)

### Key to 1X2 (match) betting odds data:

- B365H = Bet365 home win odds
- B365D = Bet365 draw odds
- B365A = Bet365 away win odds
- BSH = Blue Square home win odds
- BSD = Blue Square draw odds
- BSA = Blue Square away win odds
- BWH = Bet&Win home win odds
- BWD = Bet&Win draw odds
- BWA = Bet&Win away win odds
- GBH = Gamebookers home win odds
- GBD = Gamebookers draw odds
- GBA = Gamebookers away win odds
- IWH = Interwetten home win odds
- IWD = Interwetten draw odds
- IWA = Interwetten away win odds
- LBH = Ladbrokes home win odds
- LBD = Ladbrokes draw odds
- LBA = Ladbrokes away win odds
- PSH and PH = Pinnacle home win odds
- PSD and PD = Pinnacle draw odds
- PSA and PA = Pinnacle away win odds
- SOH = Sporting Odds home win odds
- SOD = Sporting Odds draw odds
- SOA = Sporting Odds away win odds
- SBH = Sportingbet home win odds
- SBD = Sportingbet draw odds
- SBA = Sportingbet away win odds
- SJH = Stan James home win odds
- SJD = Stan James draw odds
- SJA = Stan James away win odds
- SYH = Stanleybet home win odds
- SYD = Stanleybet draw odds
- SYA = Stanleybet away win odds
- VCH = VC Bet home win odds
- VCD = VC Bet draw odds
- VCA = VC Bet away win odds
- WHH = William Hill home win odds
- WHD = William Hill draw odds
- WHA = William Hill away win odds

## Explore the betting columns

The target is to build set of pronostics [x.yy, x.yy, x.yy] which means Home, D, Away for each betting platform.

Missing value will be filled with the line (other betting platform).

Betting score are always in 3 features which ended by:
- "H" for home
- "D" for deuce
- "A" for Away

### Extract columns from the dataset

Extract columns from dataset and build a dictionary of feature name by betting platform


In [48]:
all_betting_cols = [x for x in df.columns.tolist() if str(x).endswith("H") 
                    or str(x).endswith("D") or str(x).endswith("A")]
all_betting_cols

# from that list extract the betting platform
betting_platform={}
for v in all_betting_cols:
    bp = v[:-1]
    if bp != 'FTR_' and bp != 'HTR_' and bp != 'BbA' and bp != 'BbMxAH' and bp != 'BbAvAH':
        betting_platform[bp] = [f"{bp}H",f"{bp}D",f"{bp}A"]

betting_platform

{'B365': ['B365H', 'B365D', 'B365A'],
 'BW': ['BWH', 'BWD', 'BWA'],
 'IW': ['IWH', 'IWD', 'IWA'],
 'WH': ['WHH', 'WHD', 'WHA'],
 'VC': ['VCH', 'VCD', 'VCA'],
 'BbMx': ['BbMxH', 'BbMxD', 'BbMxA'],
 'BbAv': ['BbAvH', 'BbAvD', 'BbAvA']}

### create the new dataset
read each row of the original dataset and for the betting platform create a new row, the dictionary will be
- day features such as teams and score
- date
- betting platform odds


In [63]:
new_set = []
for idx, r in df.iterrows():
    static_variable = [r.HomeTeam, r.AwayTeam, r.Date.month]
    for key in betting_platform:
        temp = betting_platform[key]
        temp_odds =r[temp]
        new_set.append([static_variable[0],
                        static_variable[1],
                        static_variable[2],
                        temp_odds[0], temp_odds[1], temp_odds[2],
                        r.FTR]
                        )


In [64]:
bet=pd.DataFrame(new_set)
bet.shape

<IPython.core.display.Javascript object>

(53032, 7)

In [65]:
bet.isnull().sum()

0    0
1    0
2    0
3    0
4    0
5    0
6    0
dtype: int64

In [66]:
bet.dtypes

0     object
1     object
2      int64
3    float64
4    float64
5    float64
6     object
dtype: object

In [67]:
bet.sample(6)

,0,1,2,3,4,5,6
36138,Tottenham,Middlesbrough,4,1.29,6.00,13.00,H
2720,Wigan,Chelsea,8,6.50,3.80,1.65,A
43177,Blackburn,Wigan,6,1.75,3.50,4.50,H
2278,Leicester,Newcastle,12,1.63,3.75,5.80,A
46509,Stoke,Man United,1,4.00,3.40,2.00,D
2896,Norwich,West Ham,9,2.50,3.50,3.20,D


In [69]:
bet.columns=['Hteam','Ateam','Month','Odd-1','Odd-N','Odd-2','Target']
bet

,Hteam,Ateam,Month,Odd-1,Odd-N,Odd-2,Target
0,Man United,Leicester,10,1.57,3.90,7.50,H
1,Man United,Leicester,10,1.53,4.00,7.50,H
2,Man United,Leicester,10,1.55,3.80,7.00,H
3,Man United,Leicester,10,1.57,3.80,6.00,H
4,Man United,Leicester,10,1.57,4.00,7.00,H
...,...,...,...,...,...,...,...
53027,Wolves,Sunderland,5,2.20,3.20,3.00,H
53028,Wolves,Sunderland,5,2.30,3.20,3.20,H
53029,Wolves,Sunderland,5,2.30,3.25,3.20,H
53030,Wolves,Sunderland,5,2.39,3.40,3.33,H


In [71]:
result = oe.fit_transform(bet[['Target']])

In [73]:
bet['Target']=result
bet

,Hteam,Ateam,Month,Odd-1,Odd-N,Odd-2,Target
0,Man United,Leicester,10,1.57,3.90,7.50,2.0
1,Man United,Leicester,10,1.53,4.00,7.50,2.0
2,Man United,Leicester,10,1.55,3.80,7.00,2.0
3,Man United,Leicester,10,1.57,3.80,6.00,2.0
4,Man United,Leicester,10,1.57,4.00,7.00,2.0
...,...,...,...,...,...,...,...
53027,Wolves,Sunderland,5,2.20,3.20,3.00,2.0
53028,Wolves,Sunderland,5,2.30,3.20,3.20,2.0
53029,Wolves,Sunderland,5,2.30,3.25,3.20,2.0
53030,Wolves,Sunderland,5,2.39,3.40,3.33,2.0


# Statistics Descriptive

In [75]:
bet.describe()

,Month,Odd-1,Odd-N,Odd-2,Target
count,53032.000000,53032.000000,53032.000000,53032.000000,53032.000000
mean,6.701426,2.796337,4.015394,4.938389,1.173970
std,3.834360,1.960517,1.189416,4.225817,0.849623
min,1.000000,1.050000,2.500000,1.120000,0.000000
25%,3.000000,1.650000,3.300000,2.400000,0.000000
50%,8.000000,2.200000,3.550000,3.500000,1.000000
75%,10.000000,3.000000,4.200000,5.500000,2.000000
max,12.000000,28.000000,17.250000,60.000000,2.000000


In [78]:
# dealing with outliers
from sklearn.ensemble import IsolationForest
X=bet.drop(['Month','Target','Hteam','Ateam'], axis=1)

clf = IsolationForest(random_state=0).fit(X)
res=clf.predict(X)

In [83]:
outliers=pd.DataFrame(res)
outliers.columns = ['score']
outliers.value_counts()

<IPython.core.display.Javascript object>

score
 1       44626
-1        8406
dtype: int64

In [87]:
# index of outliers
to_delete=outliers.query('score < 0').index.tolist()
bet.drop(to_delete, axis=0, inplace=True)
bet.shape

(44626, 7)

In [88]:
#bet.loc[:,5] = bet.loc[:,5].map({'H':-1,'D':0,'A':1})
bet.sample(9)

,Hteam,Ateam,Month,Odd-1,Odd-N,Odd-2,Target
7329,Everton,Man City,3,3.90,3.60,2.05,2.0
10501,Sunderland,Chelsea,7,2.95,3.30,2.35,2.0
20709,West Ham,Swansea,8,2.10,3.30,3.50,2.0
14451,Bolton,Blackburn,12,2.10,3.30,3.50,2.0
37371,Everton,Aston Villa,10,2.16,3.50,4.04,1.0
3200,QPR,Everton,10,3.30,3.40,2.25,1.0
14139,Aston Villa,Man United,11,4.75,3.49,1.77,1.0
32322,West Ham,QPR,5,1.95,3.30,4.00,2.0
51099,Stoke,Wolves,10,1.90,3.36,4.06,1.0


In [89]:
# save the file
bet.to_csv('bet.csv', index=False)

In [ ]:
y

In [ ]:
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import SVC
from sklearn.svm import LinearSVC

In [ ]:
clf = OneVsRestClassifier(LinearSVC()).fit(X, y)

In [ ]:
clf.predict([[2.23,3.80,6.06,19,28], [1.23,3.80,6.06,35,16]])

In [ ]:
from sklearn.datasets import make_classification
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelBinarizer
from sklearn.neighbors import KNeighborsClassifier

In [ ]:
y_dense = LabelBinarizer().fit_transform(y)
np.unique(y_dense)

In [ ]:
np.unique(y)

In [ ]:
y

In [ ]:
Y = np.vstack((y1, y2, y3)).T

In [ ]:
forest = RandomForestClassifier(random_state=1)
multi_target_forest = MultiOutputClassifier(forest, n_jobs=-1)
multi_target_forest.fit(X, v_dense).predict_proba(X)



In [ ]:
y_dense[1100]

In [ ]:
y_dense?

In [ ]:
np.unique(y_dense)

In [ ]:
np.unique(y)

In [ ]:
y?


In [ ]:
import numpy as np
lb = LabelBinarizer()
lb.fit(np.vstack(y))

print(lb.classes_)

v_dense=lb.transform(y)
v_dense

In [ ]:
np.unique(v_dense)

In [ ]:
y

In [ ]:
np.vstack(y)

In [ ]:
y

In [ ]:
y1_dense=[]
for o in y:
    if o == 'H':
        y1_dense.append([1,0,0,0])
    elif o == 'D':
        y1_dense.append([0,1,0,0])
    else:
        y1_dense.append([0,0,1,0])
        
y1_dense

In [ ]:
forest = RandomForestClassifier(random_state=1)
multi_target_forest = MultiOutputClassifier(forest, n_jobs=-1)
multi_target_forest.fit(X, y1_dense).predict_proba(X)

